In [ ]:
%pip install --upgrade nada-dsl
%pip install --upgrade py-nillion-client
%pip install --upgrade nillion-python-helpers
%pip install python-dotenv==1.0.0

In [21]:
# %pip show nada-dsl
%pip show py-nillion-client


Name: py-nillion-client
Version: 0.6.0
Summary: Python client for Nillion network and utilities.
Home-page: 
Author: 
Author-email: 
License: Apache-2.0
Location: /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages
Requires: protobuf
Required-by: nada-numpy, nillion-python-helpers
Note: you may need to restart the kernel to use updated packages.


Import all necessary libraries

In [3]:
import py_nillion_client as nillion
from py_nillion_client import NodeKey, UserKey
from dotenv import load_dotenv
from nillion_python_helpers import get_quote_and_pay, create_nillion_client, create_payments_config
from cosmpy.aerial.client import LedgerClient
from cosmpy.aerial.wallet import LocalWallet
from cosmpy.crypto.keypairs import PrivateKey

Load the environment variables from a Devnet .env file

*Please make sure to replace the path with the correct path to your .env file*

In [ ]:
import os

home_dir = os.path.expanduser("~")
env_path = os.path.join(home_dir, ".config", "nillion", "nillion-devnet.env")

print(f"Loading environment variables from {env_path}")

load_dotenv(env_path)

for key, value in os.environ.items():
    if key.startswith("NILLION_"):
        print(f"{key}: {value}")

Read in some basic Nillion network information from the environment. 

In [5]:
cluster_id = os.getenv('NILLION_CLUSTER_ID')
chain_id = os.getenv('NILLION_NILCHAIN_CHAIN_ID')
grpc_endpoint = os.getenv('NILLION_NILCHAIN_GRPC')

print(f"Cluster ID: {cluster_id}")
print(f"Chain ID: {chain_id}")
print(f"GRPC Endpoint: {grpc_endpoint}")

Cluster ID: 9e68173f-9c23-4acc-ba81-4f079b639964
Chain ID: nillion-chain-devnet
GRPC Endpoint: localhost:26649


Each actor will need its own node key so we define a utility function here.

In [6]:
import uuid
def gen_node_key():
    return NodeKey.from_seed(uuid.uuid4().hex)

Setup for the Monadic actor

In [7]:
monadic_seed = "monadic_seed"
monadic_userkey = UserKey.from_seed(monadic_seed)
monadic_client = create_nillion_client(monadic_userkey, gen_node_key())
monadic_party_id = monadic_client.party_id
monadic_user_id = monadic_client.user_id

Setup for the Snipper actor

In [8]:
snipper_seed = "snipper_seed"
snipper_userkey = UserKey.from_seed(snipper_seed)
snipper_client = create_nillion_client(snipper_userkey, gen_node_key())
snipper_party_id = snipper_client.party_id
snipper_user_id = snipper_client.user_id

Setup for the Patient actor

In [9]:
patient_seed = "patient_seed"
patient_userkey = UserKey.from_seed(patient_seed)
patient_client = create_nillion_client(patient_userkey, gen_node_key())
patient_party_id = patient_client.party_id
patient_user_id = patient_client.user_id

Payments set up for all network actions

In [10]:
payments_config = create_payments_config(chain_id, grpc_endpoint)
payments_client = LedgerClient(payments_config)
payments_wallet = LocalWallet(
    PrivateKey(bytes.fromhex(os.getenv("NILLION_NILCHAIN_PRIVATE_KEY_0"))),
    prefix="nillion",
)

Deprecation warning: payments_endpoint should start with http:// or https://


Configuration for the program to use. Change the contents of the below cell to use another program. 

In [11]:
program_name = "double"
program_mir_path = f"binaries/double.nada.bin"

Patient Assigns Permissions to Monadic
##### The Patient stores a secret and grants Monadic the ability to modify permissions and compute on the secret.

In [23]:
program_id = f"{patient_user_id}/{program_name}"

new_secret = nillion.NadaValues(
    {
        "foo": nillion.SecretInteger(2),
    }
)

# Set permissions for the client to compute on the program
permissions = nillion.Permissions.default_for_user(patient_user_id)

# Grant Monadic permission to compute and modify permissions
permissions.add_update_permissions(set([monadic_user_id]))
permissions.add_compute_permissions({monadic_user_id: {program_id}})


# # ERROR: AttributeError: 'builtins.Permissions' object has no attribute 'allow_delegate_compute'
# permissions = nillion.Permissions.default_for_user(patient_user_id)
# permissions.allow_delegate_compute(monadic_user_id)


# # Grant Snipper permission to compute
# permissions.add_compute_permissions({snipper_user_id: {program_id}})


# Pay for and store the secret in the network and print the returned store_id
receipt_store = await get_quote_and_pay(
    patient_client,
    nillion.Operation.store_values(new_secret, ttl_days=5),
    payments_wallet,
    payments_client,
    cluster_id,
)

# Store a secret
store_id = await patient_client.store_values(
    cluster_id, new_secret, permissions, receipt_store
)

Getting quote for operation...
Quote cost is 482 unil
Submitting payment receipt 482 unil, tx hash D8D00E695B33B011CD770BE42C9C6823987149AF19A566275BAC7771015C96F3


Have Monadic run the stored program on the stored secret

In [24]:
import os

os.environ['RUST_BACKTRACE'] = 'full'

party_name = "Party1"
compute_bindings = nillion.ProgramBindings(program_id)
compute_bindings.add_input_party(party_name, monadic_party_id)
compute_bindings.add_output_party(party_name, monadic_party_id)

computation_time_secrets = nillion.NadaValues({})

# Pay for the compute
receipt_compute = await get_quote_and_pay(
    monadic_client,
    nillion.Operation.compute(program_id, computation_time_secrets),
    payments_wallet,
    payments_client,
    cluster_id,
)

# Compute on the secret
compute_id = await monadic_client.compute(
    cluster_id,
    compute_bindings,
    [store_id],
    computation_time_secrets,
    receipt_compute,
)

# 8. Return the computation result
print(f"The computation was sent to the network. compute_id: {compute_id}")
while True:
    compute_event = await monadic_client.next_compute_event()
    if isinstance(compute_event, nillion.ComputeFinishedEvent):
        print(f"✅  Compute complete for compute_id {compute_event.uuid}")
        print(f"🖥️  The result is {compute_event.result.value}")
        break

Getting quote for operation...
Quote cost is 3 unil
Submitting payment receipt 3 unil, tx hash 77154C1832C6BD7B798D7DA24F592BC77FC4DF159D14AB248C1D7D77FA293E11
The computation was sent to the network. compute_id: 1d62a176-78dc-4e78-8518-6d5f982ce971
✅  Compute complete for compute_id 1d62a176-78dc-4e78-8518-6d5f982ce971
🖥️  The result is {'my_output': 4}


~IN PROGRESS~: Have Snipper run the stored program on the stored secret

In [31]:
import os

os.environ['RUST_BACKTRACE'] = 'full'

party_name = "Party1"
compute_bindings = nillion.ProgramBindings(program_id)
compute_bindings.add_input_party(party_name, snipper_party_id)
compute_bindings.add_output_party(party_name, snipper_party_id)

########################################################################################################
# # Grant Snipper permission to compute
# permissions.add_compute_permissions({snipper_user_id: {program_id}})

# Retrieve current permissions for secret
receipt_retrieve_permissions = await get_quote_and_pay(
    patient_client,
    nillion.Operation.retrieve_permissions(),
    payments_wallet,
    payments_client,
    cluster_id,
)

# Grant Snipper permission to compute
# sidenote: this method still relies on "patient_client" instead of "monadic_client"
# to retrieve current permissions and make the update
current_permissions = await patient_client.retrieve_permissions(cluster_id, store_id, receipt_retrieve_permissions)
current_permissions.add_compute_permissions({snipper_user_id: {program_id}})
print(f"Updated permissions: {current_permissions}")


# Get cost quote, then pay for operation to update permissions
receipt_update_permissions = await get_quote_and_pay(
    monadic_client,
    nillion.Operation.update_permissions(),
    payments_wallet,
    payments_client,
    cluster_id,
)

# ComputeError: updating permissions: compute unexpected error: user does not have permissions for action
update_id = await monadic_client.update_permissions(
    cluster_id, store_id, current_permissions, receipt_update_permissions
)
########################################################################################################



computation_time_secrets = nillion.NadaValues({})

# Pay for the compute
receipt_compute = await get_quote_and_pay(
    snipper_client,
    nillion.Operation.compute(program_id, computation_time_secrets),
    payments_wallet,
    payments_client,
    cluster_id,
)

# Compute on the secret
compute_id = await snipper_client.compute(
    cluster_id,
    compute_bindings,
    [store_id],
    computation_time_secrets,
    receipt_compute,
)

# 8. Return the computation result
print(f"The computation was sent to the network. compute_id: {compute_id}")
while True:
    compute_event = await snipper_client.next_compute_event()
    if isinstance(compute_event, nillion.ComputeFinishedEvent):
        print(f"✅  Compute complete for compute_id {compute_event.uuid}")
        print(f"🖥️  The result is {compute_event.result.value}")
        break

Getting quote for operation...
Quote cost is 2 unil
Submitting payment receipt 2 unil, tx hash 8D681DDCFBD3A19FCD8E5D71FA5F43BE1C4630AB7056B6D147081F7C7993FAF9
Updated permissions: <builtins.Permissions object at 0x1065670f0>
Getting quote for operation...
Quote cost is 2 unil
Submitting payment receipt 2 unil, tx hash 923355F0E7EA0823468932C4C7AD290936D55BB4B4B4E12FF2376B8B3EEEDB3C


ComputeError: updating permissions: compute unexpected error: user does not have permissions for action